# Stage 3.1: Data Preparation for PVN3D

## 1. Identify all `models` in train/test dataset

3 folders to examine: `training_data`, `testing_data`, `testing_data_final`

In [22]:
from utils.utils_data import *

device = torch.device('cpu')
params = {'bz': 1, 'shuffle': True, 'num_workers':1}



''' Collect all meta files'''
META_FILES = []
rgb_files, depth_files, label_files, meta_files, suffixes = get_split_files('train')
META_FILES.extend(meta_files)

rgb_files, depth_files, label_files, meta_files, suffixes = get_split_files('val')
META_FILES.extend(meta_files)

meta_files = get_test_files(data_dir='./testing_data')['meta_files']
META_FILES.extend(meta_files)

meta_files = get_test_files(data_dir='./testing_data_final')['meta_files']
META_FILES.extend(meta_files)

OBJECTS = set()
for file in meta_files:
    try:
        meta = load_pickle(file)
        for item in zip(meta['object_ids'], meta['object_names']):
            OBJECTS.add(item)
    except:
        print(f'Lying test.txt. {file} not present in test dir')
        pass

print(f'Total number of objects: {len(OBJECTS)}')
print(OBJECTS)

Total number of objects: 23
{(29, 'g_lego_duplo'), (43, 'nine_hole_peg_test'), (50, 'prism'), (5, 'b_lego_duplo'), (35, 'jenga'), (51, 'pudding_box'), (55, 'sugar_box'), (21, 'e_toy_airplane'), (52, 'rubiks_cube'), (18, 'd_toy_airplane'), (56, 'tomato_soup_can'), (58, 'wood_block'), (42, 'mustard_bottle'), (14, 'cracker_box'), (26, 'foam_brick'), (39, 'master_chef_can'), (1, 'a_lego_duplo'), (48, 'potted_meat_can'), (8, 'bleach_cleanser'), (20, 'e_lego_duplo'), (57, 'tuna_fish_can'), (30, 'gelatin_box'), (13, 'c_toy_airplane')}


## 2. Perform Farthest Point Sampling on Each Object 

Compute keypoints for the 23 objects present in train/test set, using IFP (iterative farthest point) sampling.

The sampled points are saved as `.npy` files under `./models` folder. For example:

```python
    f"./models/{obj[1]}/visual_meshes/keypoints.npy"
```


In [20]:
import trimesh

n_points, n_farthest_points = int(1e4), int(1e3)

for obj in OBJECTS:     # e.g. (29, 'g_lego_duplo')
    obj_path = f'./models/{obj[1]}/visual_meshes/'
    dae_path = f'./models/{obj[1]}/visual_meshes/visual.dae'
    kpt_path = f'./models/{obj[1]}/visual_meshes/keypoints.npy' 

    loaded = trimesh.exchange.dae.load_collada(dae_path)
    mesh = trimesh.Trimesh(**list(loaded['geometry'].values())[0])

    pcd = trimesh.sample.sample_surface(mesh, n_points)[0]
    pcd = iterative_farthest_sampling(pcd, n=n_farthest_points)
    
    np.save(kpt_path, pcd)

''' ---- Visualize one farthest-sampled canonical pcd ---'''
print(os.listdir(obj_path))
with open(kpt_path,'rb') as f:
    pcd = np.load(f)
    print(pcd.shape)

pcd = trimesh.points.PointCloud(pcd, colors=np.array([1,1,1,1]*len(pcd)).reshape(-1, 4))
trimesh.scene.Scene([mesh, pcd]).show('notebook')

unable to load base texture
Traceback (most recent call last):
  File "/home/kyle/anaconda3/envs/open3d/lib/python3.9/site-packages/trimesh/exchange/dae.py", line 283, in _parse_material
    baseColorTexture = _load_texture(
  File "/home/kyle/anaconda3/envs/open3d/lib/python3.9/site-packages/trimesh/exchange/dae.py", line 267, in _load_texture
    file_data = resolver.get(file_name)
AttributeError: 'NoneType' object has no attribute 'get'
100%|██████████| 1000/1000 [00:00<00:00, 6534.40it/s]
unable to load base texture
Traceback (most recent call last):
  File "/home/kyle/anaconda3/envs/open3d/lib/python3.9/site-packages/trimesh/exchange/dae.py", line 283, in _parse_material
    baseColorTexture = _load_texture(
  File "/home/kyle/anaconda3/envs/open3d/lib/python3.9/site-packages/trimesh/exchange/dae.py", line 267, in _load_texture
    file_data = resolver.get(file_name)
AttributeError: 'NoneType' object has no attribute 'get'
100%|██████████| 1000/1000 [00:00<00:00, 6608.54it/s]
unab

['keypoints.npy', 'visual.dae', 'texture_map.png']
(1000, 3)
